# Monte Carlo

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import numpy as np

Sampling from the uniform distribution $X\sim U(a, b)$.

In [ ]:
np.random.uniform(0, 1)

Sampling from an arbitrary function $X\sim\rho$, where $\rho$ is a density defined on the finite, connected domain $(a, b)$.

In [ ]:
def sample_arbitrary_function(rho, a, b, 
                              n = 1, 
                              phi = lambda x:x,
                              sampling = np.random.uniform):
    # phi is some operation.
    
    if n == 1:
        rejected = True
        while rejected:
            
            # different sampling methods can be chosen here.
            x = sampling(a, b)
            
            # case of acceptance.
            if rho(x) >= np.random.uniform(0, 1):
                rejected = False
                return phi(x)
            
            pass
        pass
    else:
        return np.array([sample_arbitrary_function(rho, a, b, n = 1, phi = phi) for _ in range(n)])
    pass

Demonstration with the sine function, defined on $(0, \pi)$.

In [ ]:
p = lambda x:1/2*np.sin(x)

In [ ]:
sample_arbitrary_function(density, 0, np.pi)

In [ ]:
a, b = 0, np.pi
n = 10**4

solution = sample_arbitrary_function(p, a, b, n = n)

In [ ]:
plt.hist(solution, 50, density = True)

xx = np.linspace(0, np.pi, 100)
plt.plot(xx, p(xx))

plt.title('Histogram')

plt.show()

## Monte Carlo approximation

Let $x_1, \cdots, x_n\sim p$, then the sample mean $\hat{\mu}_n = \dfrac{1}{n}\sum_{i = 1}^n\phi(x_i)$ is a basic Monte Carlo estimator of $\mathbb{E}\phi(x)$.

## Importance sampling

Let $X\sim p$. Then the expectation $\mathbb{E}\phi(x) = \int_\Omega\phi(x)p(x)dx\approx\dfrac{1}{n}\sum_{i = 1}^n\phi(x_i)$. Let $q(x)$ be a proposal density such that $q(x) = 0$ if and only if $p(x) = 0$ (absolute continuity). Then $\mathbb{E}\phi(x) \approx\dfrac{1}{n}\sum_{i = 1}^n\phi(y_i)w(y_i)$, where $w(y) = \dfrac{p(y)}{q(y)}$ and $Y\sim q$.

**Principle**: choose $q(x)$ such that $q(x)\propto\vert\phi(x)\vert p(x)$, i.e. $q$ places more weight on regions where $\vert\phi(x)\vert p(x)$ is large.

However, usually we can't find such an exact $q$. Because if we did, we would have also found the partition function of $\phi(x)p(x)$, and that would be the desired expectation.

### Demonstration

From the density $p(x) = \dfrac{1}{2}\sin(x)$ we sample $X$. Let $\phi(x) = x^2$. We first find the expectation $\mathbb{E}\phi(x)$ analytically, and yield $\dfrac{1}{2}(\pi^2-4)$. Next, we sample the approximation directly.

In [ ]:
theoretical_expectation = 1/2*(np.pi**2 - 4)
print(theoretical_expectation)

In [ ]:
phi = lambda x:np.power(x, 2)

direct_sample = sample_arbitrary_function(p, a, b, n = n, phi = phi)
plt.hist(direct_sample, 50, density = True)
plt.show()

direct_sample_mean = np.average(direct_sample)

print('Sample from direct average: {}'.format(direct_sample_mean))

Let $q(x) \propto \vert\phi(x)\vert p(x) = \dfrac{x^2}{2}\sin(x)$ be a density function, then $q =\dfrac{\sin(x)}{2Z}$, where the partition function $Z = \dfrac{\pi^2 - 4}{2}$, is a legal density function. We can varify absolute continuity.

In [ ]:
q = lambda x:np.power(x, 2)/(np.power(np.pi, 2) - 4)*np.sin(x)

new_sampling = lambda a, b:sample_arbitrary_function(q, a, b)
sample = sample_arbitrary_function(q, a, b, n = n, sampling = new_sampling)

plt.hist(sample, 50, density = True)

xx = np.linspace(0, np.pi)
plt.plot(xx, q(xx))

plt.title('Map of proposal density function')
plt.show()

Recall that we defined $q(x) = Z^{-1}\phi(x)p(x)$. Therefore, $\mathbb{E}\phi(x) = \dfrac{1}{n}\sum_{i = 1}^n\phi(x_i)w(x_i) = \dfrac{\pi^2 - 4}{2}$ is trivially obtained. In this case, our sampling does not influence the outcome of the result, since $\phi w$ is a constant function.